In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


Find relevant season ID's

In [46]:
seasons_info = pd.read_csv('data/seasons_info.csv', index_col=0)

In [48]:
index_rows = []
for year in ['2020', '2021', '2022', '2023']:
    frames = [index for index, row in seasons_info.iterrows() if row['starting_at'].split('-')[0]==year]
    index_rows = index_rows + frames
index_rows

relevant_seasons = seasons_info.iloc[index_rows,:]

In [52]:
relevant_seasons.to_csv('data/analyse/relevant_seasons.csv', index=False)